In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import xgboost as xgb

In [2]:
import os
os.chdir ('/home/ubuntu/kaggle/liberty/')
os.getcwd()

'/home/ubuntu/kaggle/liberty'

In [3]:
#load train and test
train =pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [4]:
train.head()

,Hazard,T1_V1,T1_V2,T1_V3,T1_V4,T1_V5,T1_V6,T1_V7,T1_V8,T1_V9,...,T2_V6,T2_V7,T2_V8,T2_V9,T2_V10,T2_V11,T2_V12,T2_V13,T2_V14,T2_V15
Id,,,,,,,,,,,,,,,,,,,,,
1,1,15,3,2,N,B,N,B,B,D,...,2,37,1,11,6,Y,N,E,2,2
2,4,16,14,5,H,B,N,B,B,C,...,2,22,1,18,5,Y,Y,E,2,1
3,1,10,10,5,N,K,N,B,B,E,...,6,37,2,14,6,Y,Y,E,6,1
4,1,18,18,5,N,K,N,B,B,E,...,2,25,1,1,6,Y,N,C,2,6
5,1,13,19,5,N,H,N,B,B,E,...,1,22,1,2,7,N,N,E,1,1


In [5]:
train.shape

(50999, 33)

In [6]:
test.shape

(51000, 32)

In [7]:
labels = train.Hazard
train.drop('Hazard', axis=1, inplace=True)

In [8]:
columns= train.columns
test_ind = test.index

In [9]:
type(columns)

pandas.core.index.Index

In [10]:
train = np.array(train)
test = np.array(test)

In [11]:
# label encode the categorical variables
for i in range(train.shape[1]):
    if type(train[1,i]) is str:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[:,i]) + list(test[:,i]))
        train[:,i] = lbl.transform(train[:,i])
        test[:,i] = lbl.transform(test[:,i])

In [12]:
train = train.astype(float)
test = test.astype(float)

In [31]:
params = {}
params["objective"] = "reg:linear"
params["eta"] = 0.01
params["min_child_weight"] = 5
params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["colsample_bytree"] = 0.75
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 7

In [32]:
plst = list(params.items())

In [33]:
plst

[('colsample_bytree', 0.75),
 ('subsample', 0.8),
 ('min_child_weight', 5),
 ('silent', 1),
 ('eta', 0.01),
 ('objective', 'reg:linear'),
 ('scale_pos_weight', 1.0),
 ('max_depth', 7)]

In [16]:
type(plst)

list

In [17]:
#Using 5000 rows for early stopping
offset = 5000

In [18]:
num_rounds = 2000

In [19]:
xgtest = xgb.DMatrix(test)

In [20]:
xgtest

In [21]:
#create a train and validation matrices
xgtrain = xgb.DMatrix(train[offset:,:], label = labels[offset:])
xgval = xgb.DMatrix(train[:offset,:], label =labels[:offset])

In [35]:
#train using early stopping and predict
watchlist = [(xgtrain, 'train'),(xgval, 'val')]
model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds1 = model.predict(xgtest)

Will train until val error hasn't decreased in 5 rounds.
[0]	train-rmse:5.326006	val-rmse:5.230930
[1]	train-rmse:5.300186	val-rmse:5.204834
[2]	train-rmse:5.275334	val-rmse:5.179537
[3]	train-rmse:5.250468	val-rmse:5.154374
[4]	train-rmse:5.225728	val-rmse:5.129747
[5]	train-rmse:5.202209	val-rmse:5.105948
[6]	train-rmse:5.179244	val-rmse:5.082370
[7]	train-rmse:5.155847	val-rmse:5.058468
[8]	train-rmse:5.132741	val-rmse:5.035379
[9]	train-rmse:5.110453	val-rmse:5.012602
[10]	train-rmse:5.088020	val-rmse:4.989823
[11]	train-rmse:5.065714	val-rmse:4.967268
[12]	train-rmse:5.044143	val-rmse:4.945349
[13]	train-rmse:5.022717	val-rmse:4.923462
[14]	train-rmse:5.001649	val-rmse:4.902024
[15]	train-rmse:4.981421	val-rmse:4.881420
[16]	train-rmse:4.960866	val-rmse:4.860587
[17]	train-rmse:4.940622	val-rmse:4.840262
[18]	train-rmse:4.920755	val-rmse:4.820289
[19]	train-rmse:4.901200	val-rmse:4.800640
[20]	train-rmse:4.882335	val-rmse:4.781219
[21]	train-rmse:4.863327	val-rmse:4.762071
[22]	tr

In [36]:
train = train[::-1,:]
labels =labels[::-1]

In [37]:
xgbtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
xgval  = xgb.DMatrix(train[:offset,:], label=labels[:offset])

In [38]:
watchlist = [(xgtrain, 'train'), (xgval, 'val')]
model= xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=5)
preds2= model.predict(xgtest)

Will train until val error hasn't decreased in 5 rounds.
[0]	train-rmse:5.326006	val-rmse:5.269031
[1]	train-rmse:5.300186	val-rmse:5.243659
[2]	train-rmse:5.275334	val-rmse:5.219168
[3]	train-rmse:5.250506	val-rmse:5.194659
[4]	train-rmse:5.225714	val-rmse:5.170562
[5]	train-rmse:5.201183	val-rmse:5.146657
[6]	train-rmse:5.177446	val-rmse:5.123306
[7]	train-rmse:5.154194	val-rmse:5.100647
[8]	train-rmse:5.131099	val-rmse:5.078284
[9]	train-rmse:5.108123	val-rmse:5.056139
[10]	train-rmse:5.085734	val-rmse:5.034330
[11]	train-rmse:5.063492	val-rmse:5.012699
[12]	train-rmse:5.042057	val-rmse:4.991665
[13]	train-rmse:5.020544	val-rmse:4.970682
[14]	train-rmse:4.999733	val-rmse:4.950263
[15]	train-rmse:4.978916	val-rmse:4.930025
[16]	train-rmse:4.958361	val-rmse:4.910089
[17]	train-rmse:4.938204	val-rmse:4.890449
[18]	train-rmse:4.918246	val-rmse:4.870803
[19]	train-rmse:4.898612	val-rmse:4.851948
[20]	train-rmse:4.879287	val-rmse:4.833428
[21]	train-rmse:4.860535	val-rmse:4.815077
[22]	tr

In [39]:
preds = preds1 + preds2

In [40]:
type(preds)

numpy.ndarray

In [41]:
preds = pd.DataFrame({"Id": test_ind, "Hazard": preds})
preds = preds.set_index('Id')
preds.to_csv('xgboost_bm_july7th_4.csv')